In [1]:
import pandas as pd
import numpy as np

def blight_model():
    ################## Preparing training data      
    df = pd.read_csv("train.csv",index_col="ticket_id", encoding = "ISO-8859-1") 
    vars_for_model = ["fine_amount","discount_amount","judgment_amount","disposition","compliance"]
    df1= df[ vars_for_model ]
    df1.dropna(inplace=True) 
    # encoding disposition
    df1["disposition"] = df1["disposition"].astype("category")
    df1 = pd.get_dummies(df1,prefix="disposition",columns=["disposition"])
    
    #################### The model 
    from sklearn.model_selection import GridSearchCV
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import roc_auc_score
    
    target="compliance"
    features = df1.columns.tolist()
    features.remove(target)
    
    X_train = df1[features].values 
    y_train = df1[target].values

    scaler=MinMaxScaler().fit(X_train)
    X_train_scaled = scaler.transform(X_train)

    logic_reg = LogisticRegression()
    grid_values = {'C': [100, 120, 130, 140, 150, 170], 'penalty':['l1','l2']}
    grid_clf = GridSearchCV(logic_reg,param_grid=grid_values,scoring='roc_auc',n_jobs=-1)
    grid_clf.fit(X_train_scaled, y_train)

    ################### preparing test set 
    df_cv = pd.read_csv("test.csv",index_col="ticket_id", encoding = "ISO-8859-1")
    df1_cv = df_cv[ vars_for_model[:-1] ]
    df1_cv["disposition"] = df1_cv["disposition"].astype("category")
    df1_cv = pd.get_dummies(df1_cv,prefix="disposition",columns=["disposition"])
    df1_cv["fine_amount"].fillna(0)
    df1_cv = df1_cv[ features ]

    ################### predictions
    index_cv = df1_cv.index
    X_cv = df1_cv.values 
    X_cv_scaled = scaler.transform(X_cv)
    y_cv_prob   = grid_clf.predict_proba(X_cv_scaled)
    y_cv_prob   = y_cv_prob[:,1]
    
    return pd.Series(data=y_cv_prob,index=index_cv)
blight_model()

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:2828: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/lib/python3/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/lib/python3/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/home/cesarliz10/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarnin

ticket_id
284932    0.040242
285362    0.016024
285361    0.045064
285338    0.040242
285346    0.045064
285345    0.040242
285347    0.047678
285342    0.395907
285530    0.016024
284989    0.028569
285344    0.047678
285343    0.016024
285340    0.016024
285341    0.047678
285349    0.045064
285348    0.040242
284991    0.028569
285532    0.028569
285406    0.028569
285001    0.028569
285006    0.016024
285405    0.016024
285337    0.028569
285496    0.047678
285497    0.040242
285378    0.016024
285589    0.028569
285585    0.040242
285501    0.045064
285581    0.016024
            ...   
376367    0.021416
376366    0.038020
376362    0.300671
376363    0.313244
376365    0.021416
376364    0.038020
376228    0.038020
376265    0.038020
376286    0.235508
376320    0.038020
376314    0.038020
376327    0.235508
376385    0.235508
376435    0.480999
376370    0.403465
376434    0.047678
376459    0.045064
376478    0.000144
376473    0.038020
376484    0.035916
376482    0.028569
37